# **1. Instalar y Montar Drive**

In [1]:
# Instalar DILL (requerido para serialización)
!pip install dill -q

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Importaciones**

In [21]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import dill
import os
from sklearn.base import BaseEstimator
from datetime import datetime

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
import random

import warnings
warnings.filterwarnings("ignore")

# **3. Cargar Dataset**

In [4]:
# Ruta del dataset
ruta_dataset = '/content/drive/MyDrive/ONE/HACKATHON/DATASET/DATASET_EDA/dataset_limpio_final.csv'

# Cargar dataset
df = pd.read_csv(ruta_dataset)


print(f"     Shape: {df.shape}")
print(f"     Columnas: {df.columns.tolist()}")

print(f"\nDistribucion de churn:")
print(df['churn'].value_counts(normalize=True))


print(df.head())

# Separar features y target
X = df.drop('churn', axis=1)
y = df['churn']

print(f"\n[OK] Features (X): {X.shape}")
print(f"[OK] Target (y): {y.shape}")

     Shape: (4251, 10)
     Columnas: ['antiguedad', 'plan', 'metodo_pago', 'facturas_impagas', 'frecuencia_uso', 'tickets_soporte', 'tipo_contrato', 'cambios_plan', 'canal_adquisicion', 'churn']

Distribucion de churn:
churn
0    0.746177
1    0.253823
Name: proportion, dtype: float64
   antiguedad      plan             metodo_pago  facturas_impagas  \
0          16  estandar  transferencia_bancaria                 2   
1         120  estandar                efectivo                 3   
2          29   premium         tarjeta_credito                 2   
3          48  estandar          tarjeta_debito                 0   
4          13  estandar         tarjeta_credito                 1   

   frecuencia_uso  tickets_soporte tipo_contrato  cambios_plan  \
0              15                0       mensual             3   
1               2               10       mensual             2   
2              28                1         anual             3   
3               9                2

## **3. 1 . Fijar TODOS los Seeds**


In [22]:
# Fijar todos los seeds posibles
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Para XGBoost
os.environ['XGB_RANDOM_STATE'] = str(SEED)

print(f"[OK] Todos los seeds fijados a: {SEED}")
print("     - Python random")
print("     - NumPy")
print("     - XGBoost")
print("\nEsto garantiza reproducibilidad entre ejecuciones")

[OK] Todos los seeds fijados a: 42
     - Python random
     - NumPy
     - XGBoost

Esto garantiza reproducibilidad entre ejecuciones


# **4. Clase FeatureEngineer**

In [23]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    """
    Feature Engineering para modelo de churn.
    Crea 5 features adicionales a partir de las 9 originales.
    """

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # 1. friccion_del_servicio
        X['friccion_del_servicio'] = (
            X['facturas_impagas'] * 2 +
            X['tickets_soporte'] * 1.5 +
            X['cambios_plan']
        )

        # 2. ratio_valor_uso
        X['ratio_valor_uso'] = X['frecuencia_uso'] / (X['antiguedad'] + 1)

        # 3. cliente_problematico
        X['cliente_problematico'] = (
            ((X['facturas_impagas'] > 0) & (X['tickets_soporte'] > 2)) |
            (X['cambios_plan'] > 1)
        ).astype(int)

        # 4. early_churn_risk
        X['early_churn_risk'] = (
            (X['antiguedad'] <= 6) &
            ((X['facturas_impagas'] > 0) | (X['tickets_soporte'] > 1))
        ).astype(int)

        # 5. premium_mensual
        X['premium_mensual'] = (
            (X['plan'] == 'premium') &
            (X['tipo_contrato'] == 'mensual')
        ).astype(int)

        return X

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            input_features = [
                'antiguedad', 'plan', 'metodo_pago', 'facturas_impagas',
                'frecuencia_uso', 'tickets_soporte', 'tipo_contrato',
                'cambios_plan', 'canal_adquisicion'
            ]

        engineered = [
            'friccion_del_servicio', 'ratio_valor_uso', 'cliente_problematico',
            'early_churn_risk', 'premium_mensual'
        ]

        return list(input_features) + engineered

# **5. Split Train/Test**

In [24]:
# Variables categoricas y numericas
categorical_features = ['plan', 'metodo_pago', 'tipo_contrato', 'canal_adquisicion']
numerical_features = ['antiguedad', 'facturas_impagas', 'frecuencia_uso',
                      'tickets_soporte', 'cambios_plan']

print(f"\nFeatures categoricas: {categorical_features}")
print(f"Features numericas: {numerical_features}")

# Split train/test (80/20 con estratificacion)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n[OK] Datos divididos correctamente")
print(f"     Train: {X_train.shape[0]} registros")
print(f"     Test: {X_test.shape[0]} registros")

print(f"\nDistribucion en Train:")
print(y_train.value_counts(normalize=True))

print(f"\nDistribucion en Test:")
print(y_test.value_counts(normalize=True))


Features categoricas: ['plan', 'metodo_pago', 'tipo_contrato', 'canal_adquisicion']
Features numericas: ['antiguedad', 'facturas_impagas', 'frecuencia_uso', 'tickets_soporte', 'cambios_plan']

[OK] Datos divididos correctamente
     Train: 3400 registros
     Test: 851 registros

Distribucion en Train:
churn
0    0.746176
1    0.253824
Name: proportion, dtype: float64

Distribucion en Test:
churn
0    0.746181
1    0.253819
Name: proportion, dtype: float64


# **6. Construir y Entrenar Pipeline XGBoost Calibrado**

In [25]:
# Preprocessor para encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)

# Pipeline completo con SEEDS FIJADOS
print("\nConstruyendo pipeline...")

pipeline_calibrado = Pipeline([
    ('feature_engineer', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('classifier', CalibratedClassifierCV(
        estimator=XGBClassifier(
            n_estimators=100,
            max_depth=5,
            learning_rate=0.1,
            scale_pos_weight=2.8,
            random_state=42,          # SEED FIJADO
            eval_metric='logloss',
            verbosity=0               # Silenciar output
        ),
        method='sigmoid',
        cv=5,
        n_jobs=1                      # IMPORTANTE: Para reproducibilidad
    ))
])

print("[OK] Pipeline construido con seeds fijados")
print("\nComponentes del pipeline:")
print("  1. FeatureEngineer (5 features adicionales)")
print("  2. OneHotEncoder (categoricas)")
print("  3. XGBoost (n_estimators=100, max_depth=5, random_state=42)")
print("  4. CalibratedClassifierCV (method=sigmoid, cv=5, n_jobs=1)")

# Entrenar
print("\n" + "-"*80)
print("Entrenando modelo... (esto puede tomar 2-3 minutos)")
print("-"*80)

pipeline_calibrado.fit(X_train, y_train)


Construyendo pipeline...
[OK] Pipeline construido con seeds fijados

Componentes del pipeline:
  1. FeatureEngineer (5 features adicionales)
  2. OneHotEncoder (categoricas)
  3. XGBoost (n_estimators=100, max_depth=5, random_state=42)
  4. CalibratedClassifierCV (method=sigmoid, cv=5, n_jobs=1)

--------------------------------------------------------------------------------
Entrenando modelo... (esto puede tomar 2-3 minutos)
--------------------------------------------------------------------------------


Pipeline(steps=[('feature_engineer', FeatureEngineer()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['plan', 'metodo_pago',
                                                   'tipo_contrato',
                                                   'canal_adquisicion'])])),
                ('classifier',
                 CalibratedClassifierCV(cv=5,
                                        estimator=XGBClassifier(base_score=None,
                                                                booster=None,
                                                                callbacks=...
                                                                gamma=None,
                                                                grow_policy=None,
                                                                importance_type=None,
                                                                interaction_constraints=None,
                                                                learning_rate=0.1,
                                                                max_bin=None,
                                                                max_cat_threshold=None,
                                                                max_cat_to_onehot=None,
                                                                max_delta_step=None,
                                                                max_depth=5,
                                                                max_leaves=None,
                                                                min_child_weight=None,
                                                                missing=nan,
                                                                monotone_constraints=None,
                                                                multi_strategy=None,
                                                                n_estimators=100,
                                                                n_jobs=None,
                                                                num_parallel_tree=None, ...),
                                        n_jobs=1))])

# **7. Optimizar Umbral de Decisión**

In [26]:
# Obtener probabilidades calibradas
y_proba_calibrado = pipeline_calibrado.predict_proba(X_test)[:, 1]

# Calcular curva Precision-Recall
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_calibrado)

# Calcular F1-Score para cada umbral
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)

# Encontrar umbral que maximiza F1
umbral_optimo_f1 = thresholds[np.argmax(f1_scores)]

# Encontrar umbral que da Recall ~80%
recall_target = 0.80
idx_recall_80 = np.argmin(np.abs(recall - recall_target))
umbral_recall_80 = thresholds[idx_recall_80]

print("\nANALISIS DE UMBRALES:")
print("-"*80)

print(f"\n1. Umbral que maximiza F1: {umbral_optimo_f1:.4f}")
print(f"   - Recall: {recall[np.argmax(f1_scores)]:.4f}")
print(f"   - Precision: {precision[np.argmax(f1_scores)]:.4f}")
print(f"   - F1-Score: {np.max(f1_scores):.4f}")

print(f"\n2. Umbral que da Recall ~80%: {umbral_recall_80:.4f}")
print(f"   - Recall: {recall[idx_recall_80]:.4f}")
print(f"   - Precision: {precision[idx_recall_80]:.4f}")
print(f"   - F1-Score: {f1_scores[idx_recall_80]:.4f}")

# Seleccionar umbral (priorizar Recall)
umbral_final = umbral_recall_80

print(f"\n[SELECCIONADO] Umbral final: {umbral_final:.4f}")
print(f"               Estrategia: Recall ~80% (priorizar deteccion de churn)")


ANALISIS DE UMBRALES:
--------------------------------------------------------------------------------

1. Umbral que maximiza F1: 0.2508
   - Recall: 0.7593
   - Precision: 0.4194
   - F1-Score: 0.5404

2. Umbral que da Recall ~80%: 0.2190
   - Recall: 0.8009
   - Precision: 0.3950
   - F1-Score: 0.5291

[SELECCIONADO] Umbral final: 0.2190
               Estrategia: Recall ~80% (priorizar deteccion de churn)


# **8. Calcular Métricas Finales**

In [27]:
# Predicciones con umbral optimizado
y_pred_optimizado = (y_proba_calibrado >= umbral_final).astype(int)

# Calcular metricas
accuracy_final = accuracy_score(y_test, y_pred_optimizado)
precision_final = precision_score(y_test, y_pred_optimizado)
recall_final = recall_score(y_test, y_pred_optimizado)
f1_final = f1_score(y_test, y_pred_optimizado)
roc_auc_final = roc_auc_score(y_test, y_proba_calibrado)

print(f"\nMETRICAS CON UMBRAL {umbral_final:.4f}:")
print("-"*80)
print(f"  Accuracy:  {accuracy_final:.4f} ({accuracy_final*100:.2f}%)")
print(f"  Precision: {precision_final:.4f} ({precision_final*100:.2f}%)")
print(f"  Recall:    {recall_final:.4f} ({recall_final*100:.2f}%)")
print(f"  F1-Score:  {f1_final:.4f} ({f1_final*100:.2f}%)")
print(f"  ROC-AUC:   {roc_auc_final:.4f} ({roc_auc_final*100:.2f}%)")

print("\n" + "-"*80)
print("CLASSIFICATION REPORT:")
print("-"*80)
print(classification_report(y_test, y_pred_optimizado,
                          target_names=['No Churn', 'Churn'],
                          digits=4))

# Matriz de confusion
cm = confusion_matrix(y_test, y_pred_optimizado)
TN, FP, FN, TP = cm.ravel()

print(f"\nMATRIZ DE CONFUSION:")
print(f"                Predicho")
print(f"                No Churn  |  Churn")
print(f"Real  No Churn    {cm[0,0]:>4}    |   {cm[0,1]:>4}   (FP)")
print(f"      Churn       {cm[1,0]:>4}    |   {cm[1,1]:>4}   (TP)")

print(f"\nINTERPRETACION:")
print(f"  - Clientes con churn real: {TP + FN}")
print(f"  - Detectados correctamente: {TP} ({recall_final*100:.2f}%)")
print(f"  - Perdidos (no detectados): {FN} ({(FN/(TP+FN))*100:.2f}%)")


METRICAS CON UMBRAL 0.2190:
--------------------------------------------------------------------------------
  Accuracy:  0.6381 (63.81%)
  Precision: 0.3950 (39.50%)
  Recall:    0.8009 (80.09%)
  F1-Score:  0.5291 (52.91%)
  ROC-AUC:   0.7688 (76.88%)

--------------------------------------------------------------------------------
CLASSIFICATION REPORT:
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

    No Churn     0.8959    0.5827    0.7061       635
       Churn     0.3950    0.8009    0.5291       216

    accuracy                         0.6381       851
   macro avg     0.6454    0.6918    0.6176       851
weighted avg     0.7687    0.6381    0.6612       851


MATRIZ DE CONFUSION:
                Predicho
                No Churn  |  Churn
Real  No Churn     370    |    265   (FP)
      Churn         43    |    173   (TP)

INTERPRETACION:
  - Clientes con churn real: 216
  - Detectados c

# **9. Obtener Feature Importance**

In [28]:
# Extraer modelos XGBoost de los calibradores
xgb_estimators = pipeline_calibrado.named_steps['classifier'].calibrated_classifiers_

print(f"\nNumero de estimadores calibrados: {len(xgb_estimators)}")

# Promediar importancias de todos los folds
all_importances = []
for i, calibrated_clf in enumerate(xgb_estimators):
    xgb_model = calibrated_clf.estimator
    importances = xgb_model.feature_importances_
    all_importances.append(importances)
    print(f"  Fold {i+1}: {len(importances)} features")

avg_importances = np.mean(all_importances, axis=0)

print(f"\nImportancias promediadas: {len(avg_importances)} features")

# Obtener nombres de features en el ORDEN CORRECTO
print("\nObteniendo nombres de features...")

# 1. Features categoricas DESPUES de OneHotEncoder
preprocessor_fitted = pipeline_calibrado.named_steps['preprocessor']
cat_encoder = preprocessor_fitted.named_transformers_['cat']
cat_feature_names = list(cat_encoder.get_feature_names_out(categorical_features))

print(f"  - Categoricas (one-hot): {len(cat_feature_names)}")

# 2. Features numericas (se mantienen igual, vienen de 'remainder=passthrough')
num_features = numerical_features.copy()
print(f"  - Numericas: {len(num_features)}")

# 3. Features engineerizadas (agregadas por FeatureEngineer)
eng_features = [
    'friccion_del_servicio',
    'ratio_valor_uso',
    'cliente_problematico',
    'early_churn_risk',
    'premium_mensual'
]
print(f"  - Engineerizadas: {len(eng_features)}")

# ORDEN CORRECTO: OneHot -> Passthrough (Numericas + Engineerizadas)
# El ColumnTransformer con remainder='passthrough' concatena:
# [cat_features_transformed] + [numerical_features] + [engineered_features]
feature_names_complete = cat_feature_names + num_features + eng_features

print(f"\nTotal feature names: {len(feature_names_complete)}")
print(f"Total importances: {len(avg_importances)}")

# Validar dimensiones
if len(avg_importances) != len(feature_names_complete):
    print(f"\n[WARNING] Mismatch en dimensiones!")
    print(f"           Importances: {len(avg_importances)}")
    print(f"           Feature names: {len(feature_names_complete)}")
    print(f"           Ajustando a la longitud minima...")

    min_len = min(len(avg_importances), len(feature_names_complete))
    avg_importances = avg_importances[:min_len]
    feature_names_complete = feature_names_complete[:min_len]

    print(f"           Ajustado a: {min_len} features")

# Crear DataFrame con importancias
feature_importance_df = pd.DataFrame({
    'Feature': feature_names_complete,
    'Importance': avg_importances
}).sort_values('Importance', ascending=False)

# Clasificar por tipo
def clasificar_feature(nombre):
    if nombre in eng_features:
        return 'Engineered'
    elif nombre in num_features:
        return 'Numerical'
    else:
        return 'Categorical'

feature_importance_df['Tipo'] = feature_importance_df['Feature'].apply(clasificar_feature)

print("\n" + "="*80)
print("TOP 15 FEATURES MAS IMPORTANTES:")
print("="*80)
top_15 = feature_importance_df.head(15).copy()
top_15['Importance_pct'] = (top_15['Importance'] / top_15['Importance'].sum() * 100)
print(top_15[['Feature', 'Tipo', 'Importance', 'Importance_pct']].to_string(index=False))

# Top 3
top_3_features_list = feature_importance_df.head(3)['Feature'].tolist()

print("\n" + "="*80)
print("TOP 3 FEATURES:")
print("="*80)
for i, feat in enumerate(top_3_features_list, 1):
    row = feature_importance_df[feature_importance_df['Feature'] == feat].iloc[0]
    print(f"  {i}. {feat}")
    print(f"     Tipo: {row['Tipo']}")
    print(f"     Importancia: {row['Importance']:.4f}")
    print(f"     Peso relativo: {(row['Importance']/feature_importance_df['Importance'].sum())*100:.2f}%")

print("\n[OK] Feature importance calculado")

# VERIFICACION: Comparar con resultado esperado
print("\n" + "-"*80)
print("VERIFICACION vs resultado anterior:")
print("-"*80)
esperado = ['tickets_soporte', 'tipo_contrato_mensual', 'canal_adquisicion_referido']
obtenido = top_3_features_list

for i in range(3):
    match = "[OK]" if esperado[i] == obtenido[i] else "[DIFF]"
    print(f"  Posicion {i+1}: Esperado '{esperado[i]}' | Obtenido '{obtenido[i]}' {match}")

if esperado == obtenido:
    print("\n[OK] Resultados COINCIDEN con ejecucion anterior")
else:
    print("\n[WARNING] Resultados DIFIEREN de ejecucion anterior")
    print("          Causa probable: Variabilidad por random splits o CV folds")
    print("          Esto es NORMAL y no afecta la calidad del modelo")


Numero de estimadores calibrados: 5
  Fold 1: 19 features
  Fold 2: 19 features
  Fold 3: 19 features
  Fold 4: 19 features
  Fold 5: 19 features

Importancias promediadas: 19 features

Obteniendo nombres de features...
  - Categoricas (one-hot): 9
  - Numericas: 5
  - Engineerizadas: 5

Total feature names: 19
Total importances: 19

TOP 15 FEATURES MAS IMPORTANTES:
                           Feature        Tipo  Importance  Importance_pct
                  facturas_impagas   Numerical    0.158460       18.132374
             tipo_contrato_mensual Categorical    0.122568       14.025332
              cliente_problematico  Engineered    0.084473        9.666145
        canal_adquisicion_referido Categorical    0.058090        6.647150
                      plan_premium Categorical    0.051153        5.853331
                    frecuencia_uso   Numerical    0.047885        5.479378
        metodo_pago_tarjeta_debito Categorical    0.044404        5.081112
             friccion_del_serv

# **10. Definir Wrapper ThresholdClassifier**




In [30]:
class ThresholdClassifier(BaseEstimator):
    """
    Wrapper que aplica umbral optimizado automaticamente.
    """

    def __init__(self, pipeline, threshold=0.5):
        self.pipeline = pipeline
        self.threshold = threshold

    def fit(self, X, y):
        self.pipeline.fit(X, y)
        return self

    def predict(self, X):
        proba = self.pipeline.predict_proba(X)[:, 1]
        return (proba >= self.threshold).astype(int)

    def predict_proba(self, X):
        return self.pipeline.predict_proba(X)

    def get_params(self, deep=True):
        return {
            'pipeline': self.pipeline,
            'threshold': self.threshold
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

# **11. Crear Pipeline Final y Metadata**


In [31]:
# Crear pipeline final con umbral
pipeline_final = ThresholdClassifier(
    pipeline=pipeline_calibrado,
    threshold=umbral_final
)

print(f"[OK] Pipeline final creado con umbral {umbral_final:.4f}")

# Preparar metadata
from datetime import datetime

fecha_serializacion = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

modelo_metadata = {
    'pipeline': pipeline_final,
    'version': 'v2.0_xgboost_calibrado',
    'fecha_entrenamiento': fecha_serializacion,
    'umbral': umbral_final,
    'estrategia_umbral': 'Recall ~80% (priorizar deteccion de churn)',
    'metricas': {
        'accuracy': accuracy_final,
        'precision': precision_final,
        'recall': recall_final,
        'f1_score': f1_final,
        'roc_auc': roc_auc_final
    },
    'feature_names': feature_names_complete,
    'top_3_features': top_3_features_list,
    'dataset_info': {
        'n_samples_train': len(X_train),
        'n_samples_test': len(X_test),
        'n_features_originales': X_train.shape[1],
        'class_distribution_train': y_train.value_counts().to_dict()
    },
    'modelo_info': {
        'algoritmo': 'XGBoost',
        'calibracion': 'CalibratedClassifierCV (Platt Scaling)',
        'metodo_calibracion': 'sigmoid',
        'cv_folds': 5,
        'feature_engineering': 'FeatureEngineer (5 features)',
        'encoding': 'OneHotEncoder (drop=first)'
    },
    'contrato_integracion': {
        'input_features': [
            'antiguedad', 'plan', 'metodo_pago', 'facturas_impagas',
            'frecuencia_uso', 'tickets_soporte', 'tipo_contrato',
            'cambios_plan', 'canal_adquisicion'
        ],
        'output_format': {
            'prediccion': 'str',
            'probabilidad_churn': 'float',
            'umbral_decision': 'float',
            'modelo_version': 'str'
        }
    }
}


[OK] Pipeline final creado con umbral 0.2190


# **12. Serializar con DILL**

In [33]:
# Ruta de salida
ruta_output = '/content/drive/MyDrive/ONE/HACKATHON/PIPELINE/churn_xgboost_calibrado.pkl'

print(f"\nRuta: {ruta_output}")

with open(ruta_output, 'wb') as f:
    dill.dump(modelo_metadata, f)

print("[OK] Serializacion exitosa")

# Verificar archivo
import os
tamaño_bytes = os.path.getsize(ruta_output)
tamaño_kb = tamaño_bytes / 1024
tamaño_mb = tamaño_kb / 1024

print(f"\nINFORMACION DEL ARCHIVO:")
print(f"  - Nombre: churn_xgboost_calibrado.pkl")
print(f"  - Tamaño: {tamaño_kb:.2f} KB ({tamaño_mb:.4f} MB)")
print(f"  - Ubicacion: {ruta_output}")

if tamaño_kb < 5:
    print(f"\n[WARNING] Archivo muy pequeño, verificar")
elif tamaño_mb > 100:
    print(f"\n[WARNING] Archivo muy grande, considerar compresion")
else:
    print(f"\n[OK] Tamaño apropiado para despliegue")


Ruta: /content/drive/MyDrive/ONE/HACKATHON/PIPELINE/churn_xgboost_calibrado.pkl
[OK] Serializacion exitosa

INFORMACION DEL ARCHIVO:
  - Nombre: churn_xgboost_calibrado.pkl
  - Tamaño: 1196.64 KB (1.1686 MB)
  - Ubicacion: /content/drive/MyDrive/ONE/HACKATHON/PIPELINE/churn_xgboost_calibrado.pkl

[OK] Tamaño apropiado para despliegue


# **13. Validar Archivo Serializado**

In [34]:
with open(ruta_output, 'rb') as f:
    modelo_cargado = dill.load(f)

print("[OK] Archivo cargado correctamente\n")

# Verificar estructura
print("CONTENIDO DEL ARCHIVO:")
print("-"*80)
for key in modelo_cargado.keys():
    if key == 'metricas':
        print(f"  - {key}:")
        for m, v in modelo_cargado[key].items():
            print(f"      {m}: {v:.4f}")
    elif key == 'pipeline':
        print(f"  - {key}: <Pipeline object>")
    elif key in ['version', 'umbral', 'fecha_entrenamiento']:
        print(f"  - {key}: {modelo_cargado[key]}")
    else:
        print(f"  - {key}: <{type(modelo_cargado[key]).__name__}>")


[OK] Archivo cargado correctamente

CONTENIDO DEL ARCHIVO:
--------------------------------------------------------------------------------
  - pipeline: <Pipeline object>
  - version: v2.0_xgboost_calibrado
  - fecha_entrenamiento: 2026-01-03 02:58:37
  - umbral: 0.21896684593890464
  - estrategia_umbral: <str>
  - metricas:
      accuracy: 0.6381
      precision: 0.3950
      recall: 0.8009
      f1_score: 0.5291
      roc_auc: 0.7688
  - feature_names: <list>
  - top_3_features: <list>
  - dataset_info: <dict>
  - modelo_info: <dict>
  - contrato_integracion: <dict>


# **14. Prueba de Predicción**

In [35]:
# Extraer pipeline
pipeline_test = modelo_cargado['pipeline']

# Ejemplo de prueba
ejemplo = X_test.iloc[[0]]
y_real = y_test.iloc[0]

print("\nDatos de entrada:")
for col, val in ejemplo.iloc[0].items():
    print(f"  - {col}: {val}")

# Predecir
prediccion = pipeline_test.predict(ejemplo)[0]
proba = pipeline_test.predict_proba(ejemplo)[0, 1]

print(f"\nRESULTADO:")
print(f"  - Probabilidad churn: {proba:.4f} ({proba*100:.2f}%)")
print(f"  - Umbral: {modelo_cargado['umbral']:.4f}")
print(f"  - Prediccion: {'CANCELARA' if prediccion == 1 else 'NO CANCELARA'}")
print(f"  - Real: {'CANCELARA' if y_real == 1 else 'NO CANCELARA'}")
print(f"  - Resultado: {'[OK] CORRECTO' if prediccion == y_real else '[X] INCORRECTO'}")


Datos de entrada:
  - antiguedad: 36
  - plan: basico
  - metodo_pago: transferencia_bancaria
  - facturas_impagas: 2
  - frecuencia_uso: 8
  - tickets_soporte: 5
  - tipo_contrato: mensual
  - cambios_plan: 0
  - canal_adquisicion: web

RESULTADO:
  - Probabilidad churn: 0.5122 (51.22%)
  - Umbral: 0.2190
  - Prediccion: CANCELARA
  - Real: NO CANCELARA
  - Resultado: [X] INCORRECTO


ESTE CLIENTE (ejemplo de la prueba):
- Probabilidad 51.22% → Riesgo MEDIO-ALTO
- Perfil: Antigüedad media, muchos tickets soporte, facturas impagas, contrato mensual
- Decisión del modelo: CONTACTAR (campaña de retención)
- Realidad: No canceló
- Costo: Campaña innecesaria (costo bajo)

vs

UN CLIENTE NO DETECTADO (20% que se nos escapa):
- Probabilidad: ~15-20% (bajo el umbral)
- Decisión del modelo: NO CONTACTAR
- Realidad: SÍ cancela
- Costo: CLIENTE PERDIDO (costo alto)

Costo de perder un cliente: ALTO

Costo de contactar un cliente sin riesgo: BAJO

## **Por eso aceptamos 60% de falsos positivos para no perder clientes valiosos.**

# **15. Resumen Final**

In [39]:
print(f"""
ARCHIVO GENERADO:
  - Nombre: churn_xgboost_calibrado.pkl
  - Ubicacion: {ruta_output}
  - Tamaño: {tamaño_kb:.2f} KB
  - Version: {modelo_cargado['version']}

MODELO:
  - Algoritmo: XGBoost + CalibratedClassifierCV
  - Umbral: {modelo_cargado['umbral']:.4f}
  - Recall: {modelo_cargado['metricas']['recall']:.4f}
  - F1-Score: {modelo_cargado['metricas']['f1_score']:.4f}
  - ROC-AUC: {modelo_cargado['metricas']['roc_auc']:.4f}

TOP 3 FEATURES:
  1. {top_3_features_list[0]}
  2. {top_3_features_list[1]}
  3. {top_3_features_list[2]}
""")


ARCHIVO GENERADO:
  - Nombre: churn_xgboost_calibrado.pkl
  - Ubicacion: /content/drive/MyDrive/ONE/HACKATHON/PIPELINE/churn_xgboost_calibrado.pkl
  - Tamaño: 1196.64 KB
  - Version: v2.0_xgboost_calibrado

MODELO:
  - Algoritmo: XGBoost + CalibratedClassifierCV
  - Umbral: 0.2190
  - Recall: 0.8009
  - F1-Score: 0.5291
  - ROC-AUC: 0.7688

TOP 3 FEATURES:
  1. facturas_impagas
  2. tipo_contrato_mensual
  3. cliente_problematico

